In [ ]:
# Imports y carga
import pandas as pd
import geopandas as gpd
from pathlib import Path
import folium
from folium.plugins import MarkerCluster
import os, sys
import requests
from io import BytesIO
import logging  

In [ ]:

cwd = Path.cwd()
repo_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "pyproject.toml").exists():
        repo_root = p
        break

if repo_root:
    os.chdir(repo_root)
    print("Changed working dir to repo root:", repo_root)
else:
    print("Repo root with pyproject.toml not found. Current cwd remains:", cwd)
    print("You can also set repo root manually, e.g.: os.chdir('/home/tepeve/personal/repos/ba_ooh_ads')")
print("Now cwd:", Path.cwd())
print("Python executable:", sys.executable)
print(".venv exists here?:", (Path('.venv').exists()))

Changed working dir to repo root: /home/tepeve/personal/repos/ba_ooh_ads
Now cwd: /home/tepeve/personal/repos/ba_ooh_ads
Python executable: /home/tepeve/personal/repos/ba_ooh_ads/.venv/bin/python
.venv exists here?: True


In [ ]:
PROCESSED_DATA_DIR = Path("data/processed")
OUTPUT_DIR = Path("data/external")

ADS_PATH = PROCESSED_DATA_DIR / "anuncios_geolocalizados.parquet"
POPULATION_PATH = PROCESSED_DATA_DIR / "population_reach_h3.parquet"
CLUSTERS_GLOBAL_PATH = PROCESSED_DATA_DIR / "ads_clusters_global.parquet"
CLUSTERS_THEMATIC_PATH = PROCESSED_DATA_DIR / "ads_clusters_tematicos.parquet"

ZONIFICATION_PATH = OUTPUT_DIR / "zonificacion.parquet"



In [ ]:
ADS_CONSOLIDATED_PATH = PROCESSED_DATA_DIR / "tablero_anuncios_consolidado.parquet"
tablero = pd.read_parquet(ADS_CONSOLIDATED_PATH)


In [ ]:
tablero

,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,total_reach_age_55_A_59_1ring,total_reach_age_60_A_64_1ring,total_reach_age_65_A_69_1ring,total_reach_age_70_A_74_1ring,total_reach_age_75_A_79_1ring,total_reach_age_80_A_84_1ring,total_reach_age_85_A_89_1ring,total_reach_age_90_A_94_1ring,total_reach_age_95_A_99_1ring,total_reach_age_Desconocido_1ring
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1667.0,1506.0,1528.0,1314.0,979.0,601.0,335.0,159.0,31.0,828.0
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,3464.0,3120.0,2793.0,2182.0,1554.0,1053.0,551.0,268.0,56.0,3848.0
2,-58.450459,-34.599506,940638421,ACTIVO,1994-01-03,COMBINADO,FRONTAL,LUMINOSO,24,2,...,767.0,719.0,589.0,486.0,329.0,200.0,114.0,37.0,13.0,345.0
3,-58.450459,-34.599506,940638431,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,767.0,719.0,589.0,486.0,329.0,200.0,114.0,37.0,13.0,345.0
4,-58.429607,-34.615170,940638441,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,2572.0,2580.0,2288.0,1868.0,1517.0,942.0,589.0,282.0,66.0,1974.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18335,-58.385026,-34.590605,940550961,ACTIVO,1994-05-02,AVISO,SALIENTE,SIMPLE,1,2,...,1430.0,1446.0,1375.0,1367.0,1163.0,853.0,493.0,277.0,96.0,514.0
18336,-58.373533,-34.593725,940605441,ACTIVO,1994-05-23,AVISO,FRONTAL,SIMPLE,32,1,...,4696.0,3868.0,2926.0,2006.0,1360.0,845.0,532.0,237.0,66.0,6580.0
18337,-58.373146,-34.602053,940553071,ACTIVO,1994-05-03,AVISO,FRONTAL,SIMPLE,150,2,...,2950.0,2462.0,1833.0,1268.0,916.0,557.0,301.0,125.0,34.0,3468.0
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,2565.0,2272.0,2021.0,1868.0,1621.0,1051.0,581.0,254.0,73.0,1491.0


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Crear mapa centrado en Buenos Aires
m = folium.Map(location=[-34.6037, -58.3816], zoom_start=12)

# Agregar cluster de marcadores para mejor rendimiento
marker_cluster = MarkerCluster().add_to(m)

# Iterar sobre el tablero y agregar marcadores
for idx, row in tablero.iterrows():
    if pd.notna(row['lat']) and pd.notna(row['long']):
        # Crear el contenido del popup
        popup_html = f"""
        <div style="width:250px">
            <b>Clase:</b> {row.get('clase', 'N/A')}<br>
            <b>Tipo:</b> {row.get('tipo', 'N/A')}<br>
            <b>Característica:</b> {row.get('caracteristica', 'N/A')}<br>
            <hr>
            <b>Calle:</b> {row.get('calle_nombre_norm', 'N/A')} {row.get('calle_altura', '')}<br>
            <b>Barrio:</b> {row.get('nombre', 'N/A')}<br>
            <b>Distrito:</b> {row.get('distrito_desc', 'N/A')}<br>
            <hr>
            <b>Alcance Hombres:</b> {row.get('hombres_total_reach_1ring', 0):,.0f}<br>
            <b>Alcance Mujeres:</b> {row.get('mujeres_total_reach_1ring', 0):,.0f}
        </div>
        """
        
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(marker_cluster)

# Mostrar mapa
out_html = "map_tablero.html"
m.save(out_html)

from IPython.display import IFrame, display
display(IFrame(out_html, width=900, height=600))

In [ ]:
zonfica = gpd.read_parquet(ZONIFICATION_PATH)
zonfica

,distrito,area,geometry,distrito_simply,distrito_desc
0,E4 3,861946.770220,"POLYGON ((-58.45002 -34.53489, -58.45001 -34.5...",E4,EQUIPAMIENTO ESPECIAL
1,UP,93633.911434,"POLYGON ((-58.43953 -34.54007, -58.43915 -34.5...",UP,URBANIZACIÓN PARQUE
2,R2a I,167191.848540,"POLYGON ((-58.37606 -34.59239, -58.3761 -34.59...",R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA
3,UP,18519.259397,"POLYGON ((-58.45027 -34.55823, -58.45013 -34.5...",UP,URBANIZACIÓN PARQUE
4,UP,40471.046561,"POLYGON ((-58.45081 -34.53348, -58.45346 -34.5...",UP,URBANIZACIÓN PARQUE
...,...,...,...,...,...
970,UP,291606.478268,"POLYGON ((-58.40418 -34.56373, -58.40392 -34.5...",UP,URBANIZACIÓN PARQUE
971,UP,426331.238140,"POLYGON ((-58.42949 -34.55072, -58.42935 -34.5...",UP,URBANIZACIÓN PARQUE
972,UF,31166.553905,"POLYGON ((-58.36358 -34.64788, -58.36364 -34.6...",UF,URBANIZACIÓN FUTURA
973,UF,31166.553905,"POLYGON ((-58.36202 -34.64759, -58.36204 -34.6...",UF,URBANIZACIÓN FUTURA


In [ ]:
df_ads = pd.read_parquet(ADS_PATH)
df_ads


,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,needs_geocoding,h3_index,barrio_id,barrio_desc,comuna_id,comuna_desc,distrito,area,distrito_simply,distrito_desc
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,True,89c2e3103abffff,8.0,Caballito,6.0,6,C3 II,2.939484e+05,C3,CENTRO LOCAL
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,False,89c2e311e53ffff,32.0,San Nicolas,1.0,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES
2,-58.450459,-34.599506,940638421,ACTIVO,1994-01-03,COMBINADO,FRONTAL,LUMINOSO,24,2,...,False,89c2e310e53ffff,36.0,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL
3,-58.450459,-34.599506,940638431,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,False,89c2e310e53ffff,36.0,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL
4,-58.429607,-34.615170,940638441,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,False,89c2e3103cbffff,2.0,Almagro,5.0,5,C3 I,3.719011e+06,C3,CENTRO LOCAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18335,-58.385026,-34.590605,940550961,ACTIVO,1994-05-02,AVISO,SALIENTE,SIMPLE,1,2,...,False,89c2e31adafffff,29.0,Retiro,1.0,1,APH 30,5.341129e+04,APH,ÁREA DE PROTECCIÓN HISTÓRICA
18336,-58.373533,-34.593725,940605441,ACTIVO,1994-05-23,AVISO,FRONTAL,SIMPLE,32,1,...,False,89c2e31ad1bffff,29.0,Retiro,1.0,1,U13,1.160445e+05,U,URBANIZACIÓN DETERMINADA
18337,-58.373146,-34.602053,940553071,ACTIVO,1994-05-03,AVISO,FRONTAL,SIMPLE,150,2,...,False,89c2e31136fffff,32.0,San Nicolas,1.0,1,APH 51,5.644977e+05,APH,ÁREA DE PROTECCIÓN HISTÓRICA
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,False,89c2e31186bffff,21.0,Palermo,14.0,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA


In [ ]:
df_pop = pd.read_parquet(POPULATION_PATH)
df_pop

,h3_index,tramo_edad,hombres_residentes,mujeres_residentes,total_residentes,hombres_circulante,mujeres_circulante,otros_circulante,total_circulante,hombres_total_reach,mujeres_total_reach,total_reach
0,89c2e302003ffff,00 A 04,1,0,1,0,0,0,0,1,0,1
1,89c2e302003ffff,05 A 09,1,0,1,1,1,0,2,2,1,3
2,89c2e302003ffff,10 A 14,1,1,2,20,12,0,33,21,13,35
3,89c2e302003ffff,100 Y MÁS,0,0,0,1,1,0,2,1,1,2
4,89c2e302003ffff,15 A 19,1,1,2,73,48,0,122,74,49,124
...,...,...,...,...,...,...,...,...,...,...,...,...
59172,89c2e3c49a7ffff,75 A 79,1,1,2,1,2,0,3,2,3,5
59173,89c2e3c49a7ffff,80 A 84,0,1,1,1,1,0,2,1,2,3
59174,89c2e3c49a7ffff,85 A 89,0,1,1,1,0,0,1,1,1,2
59175,89c2e3c49a7ffff,90 A 94,0,0,0,1,2,0,4,1,2,4


In [ ]:
df_clusters_global = pd.read_parquet(CLUSTERS_GLOBAL_PATH)
df_clusters_global    


,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,barrio_desc,comuna_id,comuna_desc,distrito,area,distrito_simply,distrito_desc,geometry,index_right,cluster
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,Caballito,6.0,6,C3 II,2.939484e+05,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00\x8b\x17\xbc\x86\xe48M\x...,14.0,14.0
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,San Nicolas,1.0,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,0.0,0.0
2,-58.450459,-34.599506,940638421,ACTIVO,1994-01-03,COMBINADO,FRONTAL,LUMINOSO,24,2,...,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL,b'\x01\x01\x00\x00\x00r\x1d\xa7\xa2\xa89M\xc0\...,NaN,NaN
3,-58.450459,-34.599506,940638431,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL,b'\x01\x01\x00\x00\x00r\x1d\xa7\xa2\xa89M\xc0\...,NaN,NaN
4,-58.429607,-34.615170,940638441,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,Almagro,5.0,5,C3 I,3.719011e+06,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00-\xe9\xfd]\xfd6M\xc0ml\x...,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18335,-58.385026,-34.590605,940550961,ACTIVO,1994-05-02,AVISO,SALIENTE,SIMPLE,1,2,...,Retiro,1.0,1,APH 30,5.341129e+04,APH,ÁREA DE PROTECCIÓN HISTÓRICA,b'\x01\x01\x00\x00\x00M\x1d\xca\x84H1M\xc0\x83...,NaN,NaN
18336,-58.373533,-34.593725,940605441,ACTIVO,1994-05-23,AVISO,FRONTAL,SIMPLE,32,1,...,Retiro,1.0,1,U13,1.160445e+05,U,URBANIZACIÓN DETERMINADA,b'\x01\x01\x00\x00\x00N\tL\xeb\xcf/M\xc0^F[+\x...,NaN,NaN
18337,-58.373146,-34.602053,940553071,ACTIVO,1994-05-03,AVISO,FRONTAL,SIMPLE,150,2,...,San Nicolas,1.0,1,APH 51,5.644977e+05,APH,ÁREA DE PROTECCIÓN HISTÓRICA,b'\x01\x01\x00\x00\x00\xb4\x9a\x01=\xc3/M\xc0Q...,0.0,0.0
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,Palermo,14.0,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,0.0,0.0


In [ ]:
df_clusters_thematic = pd.read_parquet(CLUSTERS_THEMATIC_PATH)
df_clusters_thematic



,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,comuna_desc,distrito,area,distrito_simply,distrito_desc,geometry,index_right,macro_category_index,cluster_special,macro_category
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,6,C3 II,2.939484e+05,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00\x8b\x17\xbc\x86\xe48M\x...,18.0,2.0,8.0,daily_supply_services
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,23.0,3.0,0.0,education_and_culture
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,50.0,5.0,0.0,finance_and_corporate
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,58.0,6.0,0.0,gastronomy
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,88.0,10.0,0.0,tourism_and_entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,61.0,6.0,3.0,gastronomy
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,77.0,7.0,11.0,health_and_beauty
18339,-58.392234,-34.594160,930557141,ACTIVO,1993-06-28,AVISO,MARQUESINA/EPP,SIMPLE,2,1,...,2,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00>\xd1\xcd\xbb42M\xc0\x99...,41.0,4.0,1.0,fashion_and_clothes
18339,-58.392234,-34.594160,930557141,ACTIVO,1993-06-28,AVISO,MARQUESINA/EPP,SIMPLE,2,1,...,2,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00>\xd1\xcd\xbb42M\xc0\x99...,58.0,6.0,0.0,gastronomy


In [ ]:
radios_censales = pd.read_parquet(OUTPUT_DIR / "radios_censales.parquet")
radios_censales

,jur,dpto,cod_indec,geometry
24,Ciudad Autónoma de Buenos Aires,Comuna 11,020771804,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x13\x00...
55,Ciudad Autónoma de Buenos Aires,Comuna 11,020771807,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x12\x00...
97,Ciudad Autónoma de Buenos Aires,Comuna 11,020771010,"b'\x01\x03\x00\x00\x00\x01\x00\x00\x00""\x00\x0..."
98,Ciudad Autónoma de Buenos Aires,Comuna 11,020771011,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\t\x00\x..."
105,Ciudad Autónoma de Buenos Aires,Comuna 11,020771801,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x11\x00...
...,...,...,...,...
66479,Ciudad Autónoma de Buenos Aires,Comuna 11,020771805,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\x...
66493,Ciudad Autónoma de Buenos Aires,Comuna 6,020420402,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x11\x00...
66500,Ciudad Autónoma de Buenos Aires,Comuna 6,020420407,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x10\x00...
66501,Ciudad Autónoma de Buenos Aires,Comuna 6,020420409,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x10\x00...


In [ ]:
gdf_zonif= pd.read_parquet(ZONIFICATION_PATH)
gdf_zonif.columns

Index(['distrito', 'area', 'geometry', 'distrito_simply', 'distrito_desc'], dtype='object')

In [ ]:
gdf_zonif.columns

Index(['distrito', 'area', 'geometry', 'distrito_simply', 'distrito_desc'], dtype='object')

In [ ]:
df_ads

,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,needs_geocoding,h3_index,barrio_id,barrio_desc,comuna_id,comuna_desc,distrito,area,distrito_simply,distrito_desc
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,True,89c2e3103abffff,8.0,Caballito,6.0,6,C3 II,2.939484e+05,C3,CENTRO LOCAL
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,False,89c2e311e53ffff,32.0,San Nicolas,1.0,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES
2,-58.450459,-34.599506,940638421,ACTIVO,1994-01-03,COMBINADO,FRONTAL,LUMINOSO,24,2,...,False,89c2e310e53ffff,36.0,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL
3,-58.450459,-34.599506,940638431,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,False,89c2e310e53ffff,36.0,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL
4,-58.429607,-34.615170,940638441,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,False,89c2e3103cbffff,2.0,Almagro,5.0,5,C3 I,3.719011e+06,C3,CENTRO LOCAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18335,-58.385026,-34.590605,940550961,ACTIVO,1994-05-02,AVISO,SALIENTE,SIMPLE,1,2,...,False,89c2e31adafffff,29.0,Retiro,1.0,1,APH 30,5.341129e+04,APH,ÁREA DE PROTECCIÓN HISTÓRICA
18336,-58.373533,-34.593725,940605441,ACTIVO,1994-05-23,AVISO,FRONTAL,SIMPLE,32,1,...,False,89c2e31ad1bffff,29.0,Retiro,1.0,1,U13,1.160445e+05,U,URBANIZACIÓN DETERMINADA
18337,-58.373146,-34.602053,940553071,ACTIVO,1994-05-03,AVISO,FRONTAL,SIMPLE,150,2,...,False,89c2e31136fffff,32.0,San Nicolas,1.0,1,APH 51,5.644977e+05,APH,ÁREA DE PROTECCIÓN HISTÓRICA
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,False,89c2e31186bffff,21.0,Palermo,14.0,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA


In [ ]:
df_ads.columns

Index(['long', 'lat', 'nro_anuncio', 'estado_anuncio', 'fecha_alta_anuncio',
       'clase', 'tipo', 'caracteristica', 'metros', 'zona', 'calle_nombre',
       'calle_altura', 'direccion', 'ciudad', 'pais', 'calle_nombre_norm',
       'full_address', 'needs_geocoding', 'h3_index', 'barrio_id',
       'barrio_desc', 'comuna_id', 'comuna_desc', 'distrito', 'area',
       'distrito_simply', 'distrito_desc'],
      dtype='object')

In [ ]:
df_pop

,h3_index,tramo_edad,hombres_residentes,mujeres_residentes,total_residentes,hombres_circulante,mujeres_circulante,otros_circulante,total_circulante,hombres_total_reach,mujeres_total_reach,total_reach
0,89c2e302003ffff,00 A 04,1,0,1,0,0,0,0,1,0,1
1,89c2e302003ffff,05 A 09,1,0,1,1,1,0,2,2,1,3
2,89c2e302003ffff,10 A 14,1,1,2,20,12,0,33,21,13,35
3,89c2e302003ffff,100 Y MÁS,0,0,0,1,1,0,2,1,1,2
4,89c2e302003ffff,15 A 19,1,1,2,73,48,0,122,74,49,124
...,...,...,...,...,...,...,...,...,...,...,...,...
59172,89c2e3c49a7ffff,75 A 79,1,1,2,1,2,0,3,2,3,5
59173,89c2e3c49a7ffff,80 A 84,0,1,1,1,1,0,2,1,2,3
59174,89c2e3c49a7ffff,85 A 89,0,1,1,1,0,0,1,1,1,2
59175,89c2e3c49a7ffff,90 A 94,0,0,0,1,2,0,4,1,2,4


In [ ]:
df_pop.columns

Index(['h3_index', 'tramo_edad', 'hombres_residentes', 'mujeres_residentes',
       'total_residentes', 'hombres_circulante', 'mujeres_circulante',
       'otros_circulante', 'total_circulante', 'hombres_total_reach',
       'mujeres_total_reach', 'total_reach'],
      dtype='object')

In [ ]:
df_clusters_global

,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,barrio_desc,comuna_id,comuna_desc,distrito,area,distrito_simply,distrito_desc,geometry,index_right,cluster
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,Caballito,6.0,6,C3 II,2.939484e+05,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00\x8b\x17\xbc\x86\xe48M\x...,14.0,14.0
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,San Nicolas,1.0,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,0.0,0.0
2,-58.450459,-34.599506,940638421,ACTIVO,1994-01-03,COMBINADO,FRONTAL,LUMINOSO,24,2,...,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL,b'\x01\x01\x00\x00\x00r\x1d\xa7\xa2\xa89M\xc0\...,NaN,NaN
3,-58.450459,-34.599506,940638431,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,Villa Crespo,15.0,15,E3,6.875788e+04,E3,EQUIPAMIENTO LOCAL,b'\x01\x01\x00\x00\x00r\x1d\xa7\xa2\xa89M\xc0\...,NaN,NaN
4,-58.429607,-34.615170,940638441,ACTIVO,1994-01-03,COMBINADO,SALIENTE,LUMINOSO,2,2,...,Almagro,5.0,5,C3 I,3.719011e+06,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00-\xe9\xfd]\xfd6M\xc0ml\x...,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18335,-58.385026,-34.590605,940550961,ACTIVO,1994-05-02,AVISO,SALIENTE,SIMPLE,1,2,...,Retiro,1.0,1,APH 30,5.341129e+04,APH,ÁREA DE PROTECCIÓN HISTÓRICA,b'\x01\x01\x00\x00\x00M\x1d\xca\x84H1M\xc0\x83...,NaN,NaN
18336,-58.373533,-34.593725,940605441,ACTIVO,1994-05-23,AVISO,FRONTAL,SIMPLE,32,1,...,Retiro,1.0,1,U13,1.160445e+05,U,URBANIZACIÓN DETERMINADA,b'\x01\x01\x00\x00\x00N\tL\xeb\xcf/M\xc0^F[+\x...,NaN,NaN
18337,-58.373146,-34.602053,940553071,ACTIVO,1994-05-03,AVISO,FRONTAL,SIMPLE,150,2,...,San Nicolas,1.0,1,APH 51,5.644977e+05,APH,ÁREA DE PROTECCIÓN HISTÓRICA,b'\x01\x01\x00\x00\x00\xb4\x9a\x01=\xc3/M\xc0Q...,0.0,0.0
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,Palermo,14.0,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,0.0,0.0


In [ ]:
df_clusters_global.columns

Index(['long', 'lat', 'nro_anuncio', 'estado_anuncio', 'fecha_alta_anuncio',
       'clase', 'tipo', 'caracteristica', 'metros', 'zona', 'calle_nombre',
       'calle_altura', 'direccion', 'ciudad', 'pais', 'calle_nombre_norm',
       'full_address', 'needs_geocoding', 'h3_index', 'barrio_id',
       'barrio_desc', 'comuna_id', 'comuna_desc', 'distrito', 'area',
       'distrito_simply', 'distrito_desc', 'geometry', 'index_right',
       'cluster'],
      dtype='object')

In [ ]:
df_clusters_thematic

,long,lat,nro_anuncio,estado_anuncio,fecha_alta_anuncio,clase,tipo,caracteristica,metros,zona,...,comuna_desc,distrito,area,distrito_simply,distrito_desc,geometry,index_right,macro_category_index,cluster_special,macro_category
0,-58.444474,-34.608641,940631631,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,6,C3 II,2.939484e+05,C3,CENTRO LOCAL,b'\x01\x01\x00\x00\x00\x8b\x17\xbc\x86\xe48M\x...,18.0,2.0,8.0,daily_supply_services
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,23.0,3.0,0.0,education_and_culture
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,50.0,5.0,0.0,finance_and_corporate
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,58.0,6.0,0.0,gastronomy
1,-58.389475,-34.601973,940631641,ACTIVO,1994-09-29,COMBINADO,MARQUESINA/EPP,LUMINOSO,1,2,...,1,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00o\xc8\x83S\xda1M\xc0)S\x...,88.0,10.0,0.0,tourism_and_entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,61.0,6.0,3.0,gastronomy
18338,-58.410795,-34.580658,940551401,ACTIVO,1994-04-29,AVISO,FRONTAL,SIMPLE,1,1,...,14,R2a I,6.642642e+05,R2a,RESIDENCIAL GENERAL DE DENSIDAD ALTA,b'\x01\x01\x00\x00\x00\x05\xea\x93\xee\x944M\x...,77.0,7.0,11.0,health_and_beauty
18339,-58.392234,-34.594160,930557141,ACTIVO,1993-06-28,AVISO,MARQUESINA/EPP,SIMPLE,2,1,...,2,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00>\xd1\xcd\xbb42M\xc0\x99...,41.0,4.0,1.0,fashion_and_clothes
18339,-58.392234,-34.594160,930557141,ACTIVO,1993-06-28,AVISO,MARQUESINA/EPP,SIMPLE,2,1,...,2,C2,1.494189e+06,C2,CENTROS PRINCIPALES,b'\x01\x01\x00\x00\x00>\xd1\xcd\xbb42M\xc0\x99...,58.0,6.0,0.0,gastronomy


In [ ]:
df_clusters_thematic.columns

Index(['long', 'lat', 'nro_anuncio', 'estado_anuncio', 'fecha_alta_anuncio',
       'clase', 'tipo', 'caracteristica', 'metros', 'zona', 'calle_nombre',
       'calle_altura', 'direccion', 'ciudad', 'pais', 'calle_nombre_norm',
       'full_address', 'needs_geocoding', 'h3_index', 'barrio_id',
       'barrio_desc', 'comuna_id', 'comuna_desc', 'distrito', 'area',
       'distrito_simply', 'distrito_desc', 'geometry', 'index_right',
       'macro_category_index', 'cluster_special', 'macro_category'],
      dtype='object')